In [15]:
from modeling.gen6.api import symbolic, exp
from modeling.gen7.api7 import MFunctionalSet, MFunctionalSetLeaf
from engine.torchdata import load_vals, print_formatted_table
import numpy as np
import torch
from scipy import optimize

In [2]:
x1, x2, x3, u1, u2 = symbolic('x1', 'x2', 'x3', 'u1', 'u2')

discipline1 = MFunctionalSetLeaf(
    u1 == x1**2 + x2 + x3 - 0.2*u2
)
discipline2 = MFunctionalSetLeaf(
    u2 == u1**0.5 + x1 + x2
)
MDA = MFunctionalSet(discipline1, discipline2)
FPF = MDA.subsetof(
    0 <= x1, x1 <= 10,
    0 <= x2, x2 <= 10,
    0 <= x3, x3 <= 10,
    3.16 <= u1,
    u2 <= 24
).minimize(
    x3**2 + x2 + u1 + exp(-u2)
)
FPF_AAO = FPF.config(residuals=[discipline1, discipline2])

In [41]:
obj, dobj, xguess, cons, idxs, solidxs = FPF_AAO.build_opt(
    x0={'x1': 5, 'x2': 2, 'x3': 1, 'u1': 1, 'u2': 1})

In [42]:
xsol = optimize.minimize(obj, xguess, jac=dobj, 
                         constraints=cons, method='SLSQP')

In [43]:
xsol.fun

3.183393951640044

In [44]:
idxrev = {var.item():key for key,var in idxs.items()}
print_formatted_table([np.round(xsol.x[np.argsort(solidxs)],2)], 
                      idxs, idxrev)

x3 u1   x2 x1   u2  
0  3.16 0  1.98 3.76
